<a id="top"></a>

The aim of this notebook is to clean and prepare the data in the leavetimes dataset

***

# Import Packages

In [1]:
import pandas as pd

***

# Load Original Data File

In [ ]:
# load in original data file
# this may take a couple of minutes and can result in a memory error if many notebooks are in use
t1 = datetime.datetime.now()
df_leavetimes = pd.read_csv('/home/faye/data/rt_leavetimes_DB_2018.txt', sep=';',error_bad_lines=False)
diff = datetime.datetime.now() - t1
print(f"Time to read: {diff}")

***

<a id="contents"></a>
# Contents

- [1. Data Quality Report](#data_quality_report)
    - [1.1. Overview of the Dataset](#overview)
    - [1.2. Feature Data Types](#feature_data_types)
    - [1.3. Duplicates and Constants](#duplicates_and_constants)
    - [1.4. Descriptive Statistics for Continuous Features](#desc_stats_continuous)
    - [1.5. Descriptive Statistics for Categorical Features](#desc_stats_categorical)
    - [1.6. Plotting Continuous Features](#plotting_continuous)
    - [1.7. Plotting Categorical Features](#plotting_categorical)
    - [1.8. Logical Integrity Check](#logical_integrity_check)
- [2. Data Quality Plan](#data_quality_plan)
    - [2.1. The Plan](#the_plan)
    - [2.2. Applying the Solutions](#applying_solutions)

***

<a id="data_quality_report"></a>
# 1. Data Quality Report
[Back to contents](#contents)

<a id="overview"></a>
## 1.1. Overview of the Dataset
[Top of section](#data_quality_report)

<a id="feature_data_types"></a>
## 1.2. Feature Data Types
[Top of section](#data_quality_report)

<a id="duplicates_and_constants"></a>
## 1.3. Duplicates and Constants
[Top of section](#data_quality_report)

<a id="desc_stats_continuous"></a>
## 1.4. Descriptive Statistics for Continuous Features
[Top of section](#data_quality_report)

<a id="desc_stats_categorical"></a>
## 1.5. Descriptive Statistics for Categorical Features
[Top of section](#data_quality_report)

<a id="plotting_continuous"></a>
## 1.6. Plotting Continuous Features
[Top of section](#data_quality_report)

<a id="plotting_categorical"></a>
## 1.7. Plotting Categorical Features
[Top of section](#data_quality_report)

<a id="logical_integrity_check"></a>
## 1.8. Logical Integrity Check
[Top of section](#data_quality_report)

<a id="data_quality_plan"></a>
# 2. Data Quality Plan
[Back to contents](#contents)

<a id="the_plan"></a>
## 2.1. The Plan
[Top of section](#data_quality_plan)

<a id="applying_solutions"></a>
## 2.2. Applying the Solutions
[Top of section](#data_quality_plan)

***

[Back to top](#top)